In [ ]:
from collections import defaultdict as ddict
from typing import DefaultDict, Set
import itertools

import freeman as fm

import pandas as pd
import numpy as np
import networkx as nx

In [ ]:
bruno = pd.read_csv(
    "csv/players.csv",
     usecols=("date", "player_name", "team", "player_id", "match_id", "event_id", "rating"),
)

In [ ]:
bruno.loc[:, "date"] = pd.to_datetime(bruno["date"], format="%Y-%m-%d")
bruno.loc[:, "player_name"] = bruno["player_name"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)
bruno = bruno[bruno["date"] >= "2018-01-01"]

In [ ]:
bruno.cabeca = bruno.head
bruno.cabeca()

In [ ]:
upid_rating = bruno.groupby(["player_id", "player_name"])["rating"]
median = upid_rating.median()
median.name = "median"
mean = upid_rating.mean()
mean.name = "mean"

helionp = pd.concat([median, mean], axis=1)
helionp.ceara = helionp.head
helionp.ceara()

In [ ]:
teams: DefaultDict[str, Set[str]] = ddict(set)
_ = bruno.groupby(["team", "match_id"])["player_id"].apply(
    lambda x: teams[x.name[0]].update(x.values)
)

In [ ]:
with open("csgo.gml", "w+") as f:
    f.write("graph [\n    directed 0\n")
    for (p_id, p_name), (median, mean) in helionp.iterrows():
        f.write("    node [\n")
        f.write(f"        id {p_id}\n")
        f.write(f"        label \"{p_name}\"\n")
        f.write("    ]\n")
    edges = set()
    for team in teams:
        for edge in itertools.combinations(teams[team], 2):
            edge = tuple(sorted(edge)) # Probably noy necessary
            if edge not in edges:
                edges.add(edge)

                f.write("    edge [\n")
                f.write(f"        source {edge[0]}\n")
                f.write(f"        target {edge[1]}\n")
                f.write("    ]\n")
    f.write("]\n")

In [ ]:
g = fm.load('csgo.gml')

In [ ]:
g.set_all_nodes(size=10, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.25))

In [ ]:
g.move("kamada_kawai")

In [ ]:
g.draw()

In [ ]:
print(bruno["player_name"].unique().shape)
print(bruno["player_id"].unique().shape)

In [ ]:
s = set()
for i in helionp.index.values:
#     print(i)
    if i[0] in s:
        print(i)
    else:
        s.add(i[0])
#     if helionp.loc[i[0]].shape[0] != 1:
#         print(i)